# Forecasting Data #

In [1]:
# imports
import pandas as pd
import numpy as np
import time

In [2]:
# define data location
#location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
location = '/Users/loki/Documents/Data/Forecasting Project/'

## Data Cleaning & Checking 

In [ ]:
# hourly data
use_oct_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.10.xlsx')
use_nov_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.11.xlsx') 
use_dec_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.12.xlsx') 
use_jan_in = pd.read_excel(location+'Zip_HourlylUsage_2018.01.xlsx')
use_feb_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.02.xlsx')
use_mar_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.03.xlsx')
use_apr_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.04.xlsx')
use_may_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.05.xlsx')
use_jun_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.06.xlsx')
use_jul_in = pd.read_excel(location+'Zip_HourlylUsage_2018.07.xlsx')
use_aug_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.08.xlsx')
use_sep_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.09.xlsx')

use_oct_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2016.10.xlsx')
use_nov_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2016.11.xlsx') 
use_dec_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2016.12.xlsx') 
use_jan_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.01.xlsx')
use_feb_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.02.xlsx')
use_mar_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.03.xlsx')
use_apr_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.04.xlsx')
use_may_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.05.xlsx')
use_jun_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.06.xlsx')
use_jul_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.07.xlsx')
use_aug_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.08.xlsx')
use_sep_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.09.xlsx')

In [4]:
# merge use data
hourly_in = [use_oct_in, use_nov_in, use_dec_in, use_jan_in, use_feb_in, use_mar_in, 
             use_apr_in, use_may_in, use_jun_in, use_jul_in, use_aug_in, use_sep_in,
             use_oct_in2, use_nov_in2, use_dec_in2, use_jan_in2, use_feb_in2, use_mar_in2, 
             use_apr_in2, use_may_in2, use_jun_in2, use_jul_in2, use_aug_in2, use_sep_in2]
use = pd.concat(hourly_in)


In [5]:
# create ID tuple
ids = pd.Series(list(map(tuple, use[['DACCOUNTID', 'DMETERNO']].values)))
use['ID'] = ids.values

ids = pd.Series(list(map(tuple, customer_in[['DACCOUNTID', 'DMETERNO']].values)))
customer_in['ID'] = ids.values

In [6]:
# find gas records
use = use.loc[use['UOM'] == 'CCF']
use = use.drop(columns=['UOM'])

customer = customer_in.drop(columns=['CITY', 'STATE', 'ZIPCODE', 'COUNTYCODE'])

# convert to datetime
use['Dt'] =  pd.to_datetime(use['METERREADDATE'])

## Data Restructuring ##

In [7]:
# functions for naming consistency
def decrement(x, startswith, split):
    """
    decrements a passed string of form "demo#" by 1
    
    Parameters
    ----------
    x : string to be decremented
    split : string to split on

    Returns
    ----------
    y : decremented string
    """
    if x.startswith(startswith):
        a,b = x.split(split)
        b = int(b)-1
        y = a + split + str(b)

        return y

    else:
        return x

    
def interval_to_hour(df):
    """
    function for fast rename/relabel during tidying process
    
    Parameters
    ----------
    df : pandas data frame

    Returns
    ----------
    df : data frame with updated column names
    """
    
    df = df.rename(columns=lambda x: decrement(x, "INTERVAL_", "_"))
    df = df.rename(columns=lambda x: x.replace("INTERVAL_", "HR"))
    return df

In [8]:
# rename for consistency
use = interval_to_hour(use)
use = use.drop(columns=['METERREADDATE','HR24'])

In [9]:
# Tidy / Stack data (transform into tall data - one row per customer per hour):
# ref: http://www.jeannicholashould.com/tidy-data-in-python.html
tidy_use = pd.melt(use, 
                   id_vars=['ID','DACCOUNTID','DMETERNO','Dt'],
                   var_name='Hour', value_name='Use')


In [10]:
# relabel/retype
tidy_use['Hour'] = tidy_use['Hour'].str.extract('(\d+)').astype(int)

In [11]:
# append rate code data
tidy_use = tidy_use.merge(customer, how='inner', on=['ID','DACCOUNTID','DMETERNO'])
#tidy_use.head()

### Make daily data ###

In [ ]:
# for daily aggregation
daily_use = tidy_use.groupby(['ID','DACCOUNTID','DMETERNO','Dt']).sum().reset_index()

# Add dummy variables for day-of-week
daily_use['Weekday'] = pd.get_dummies(daily_use['Dt'].dt.weekday < 5)[True]

# Add dummy variables for holidays
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
dr = pd.date_range(start=min(daily_use['Dt']), end=max(daily_use['Dt']))
holidays = cal.holidays(start=dr.min(), end=dr.max())

daily_use['Holiday'] = pd.get_dummies(daily_use['Dt'].isin(holidays))[True]

In [ ]:
# find IDs with at least 720 days of data
ids_to_plot = daily_use.groupby('ID').count()
sufficient_ids = ids_to_plot[ids_to_plot['Use'] >= 720].reset_index()['ID']
print(f"{100*len(sufficient_ids)/len(use['ID'].unique())} percent of IDs have at least 720 days of data")

In [ ]:
# find the dates for which sufficient_ids all have data
pivoted = (daily_use[daily_use['ID'].isin(sufficient_ids)]
           .pivot_table(index='Dt', columns=['ID'], values='Use', fill_value=None))
sufficient_dates = pivoted.dropna().reset_index()['Dt']
print(f"The IDs with sufficient data overlap on {len(sufficient_dates)} days of the most recent year of data")

In [ ]:
# filter total use dataset to contain only IDs who have sufficient data and only dates where sufficient_ids have data
sufficient_use = daily_use[(daily_use['ID'].isin(sufficient_ids)) & (daily_use['Dt'].isin(sufficient_dates))]

In [ ]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
# data.to_csv(location+'peco.csv', sep='\t')
daily_use.to_pickle(location+'fcast_daily.pkl.zip')
sufficient_use.to_pickle(location+'fcast_sufficient_daily.pkl.zip')
ids.to_pickle(location+'fcast_daily_ids.pkl.zip')
sufficient_ids.to_pickle(location+'fcast_sufficient_daily_ids.pkl.zip')

### Make hourly data###

In [12]:
# create single column for date and time
tidy_use['year'] = pd.to_datetime(tidy_use['Dt'].values).year
tidy_use['month'] = pd.to_datetime(tidy_use['Dt'].values).month
tidy_use['day'] = pd.to_datetime(tidy_use['Dt'].values).day
tidy_use['datetime'] = pd.to_datetime(tidy_use[['year','month','day','Hour']])

In [13]:
# Add dummy variables for day-of-week
#use = use.join(pd.get_dummies(use['Dt'].dt.weekday_name))
tidy_use['Weekday'] = pd.get_dummies(tidy_use['Dt'].dt.weekday < 5)[True]

# Add dummy variables for hour-of-day
tidy_use = tidy_use.join(pd.get_dummies(tidy_use['Hour'],prefix='HR'))

# Add dummy variables for part-of-day
tidy_use['AM'] = tidy_use[(tidy_use[Dt].dt.hour >= 5) & (tidy_use[Dt].dt.hour < 9)] # 6-9 am
tidy_use['DAY'] = tidy_use[(tidy_use[Dt].dt.hour >= 9) & (tidy_use[Dt].dt.hour < 16)] # 9 am-5 pm
tidy_use['EVE'] = tidy_use[(tidy_use[Dt].dt.hour >= 16) & (tidy_use[Dt].dt.hour < 21)] # 5-10 pm
tidy_use['NIGHT'] = tidy_use[(tidy_use[Dt].dt.hour >= 21) & (tidy_use[Dt].dt.hour < 5)] # 11 pm-6 am

# Add dummy variables for holidays
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
dr = pd.date_range(start=min(tidy_use['Dt']), end=max(tidy_use['Dt']))
holidays = cal.holidays(start=dr.min(), end=dr.max())

tidy_use['Holiday'] = pd.get_dummies(tidy_use['Dt'].isin(holidays))[True]
#tidy_use = tidy_use.rename(columns={True:'Holiday'})
#tidy_use = tidy_use.drop(columns=[False])
#tidy_use.head()

In [14]:
# sort by date & time
tidy_use = tidy_use.drop(columns=['Dt','year','month','day','Hour'])
tidy_use = tidy_use.sort_values(by=['datetime'])
tidy_use = tidy_use.rename(columns={'datetime':'Dt'}) # for consistency
#tidy_use.head()

In [ ]:
# find IDs with at least 720 days of data
ids_to_plot = tidy_use.groupby('ID').count()
sufficient_ids = ids_to_plot[ids_to_plot['Use'] >= 720].reset_index()['ID']
print(f"{100*len(sufficient_ids)/len(use['ID'].unique())} percent of IDs have at least 720 days of data")

In [ ]:
# find the dates for which sufficient_ids all have data
pivoted = (tidy_use[tidy_use['ID'].isin(sufficient_ids)]
           .pivot_table(index='Dt', columns=['ID'], values='Use', fill_value=None))
sufficient_dates = pivoted.dropna().reset_index()['Dt']
print(f"The IDs with sufficient data overlap on {len(sufficient_dates)} days of the most recent year of data")

In [ ]:
# filter total use dataset to contain only IDs who have sufficient data and only dates where sufficient_ids have data
sufficient_use = tidy_use[(tidy_use['ID'].isin(sufficient_ids)) & (tidy_use['Dt'].isin(sufficient_dates))]

### Save ###

In [ ]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
# data.to_csv(location+'peco.csv', sep='\t')
tidy_use.to_pickle(location+'fcast_hourly.pkl.zip')
sufficient_use.to_pickle(location+'fcast_sufficient_hourly.pkl.zip')
ids.to_pickle(location+'fcast_daily_ids.pkl.zip')
sufficient_ids.to_pickle(location+'fcast_sufficient_hourly_ids.pkl.zip')

 ### Merge use & weather? ###

In [4]:
# load data
hourly = pd.read_pickle(location+'peco_hourly.pkl.zip')
sufficient_hourly = pd.read_pickle(location+'peco_sufficient_hourly.pkl.zip')
weather = pd.read_pickle(location+'hourly_weather.pkl.zip')

In [ ]:
# merge
hourly = pd.merge(hourly, weather, how='inner', on=['Dt'])
sufficient_hourly = pd.merge(sufficient_hourly, weather, how='inner', on=['Dt'])
# CustIDs || Date | Consumption |||| Weather_variables 